In [3]:
import numpy as np
import pandas as pd

import lightgbm
import matplotlib.pylab as plt
import os

from sklearn.model_selection import train_test_split
import sklearn

Inserting all features from test into one dataframe

In [4]:
folder_train = '../features_train_in_use'

features_train = pd.read_csv(os.path.join(folder_train, os.listdir(folder_train)[0]))
for i in range(1,len(os.listdir(folder_train))):
    f = os.path.join(folder_train,os.listdir(folder_train)[i])
    if os.path.isfile(f):
        features_train = pd.concat([features_train, pd.read_csv(f)], axis=1)
print(features_train)
        
        

       plaace_h_1  plaace_h_2  plaace_h_3  plaace_h_4  features
0               1           1           1           0         0
1               1           1           1           0         0
2               1           1           1           0         0
3               1           1           1           0         0
4               1           1           1           0         0
...           ...         ...         ...         ...       ...
12854           2           8          11           2        90
12855           2           8          11           2        90
12856           2           8          11           2        90
12857           2           8          11           2        90
12858           2           8          11           2        90

[12859 rows x 5 columns]


Repeat for test features

In [5]:
folder_test = '../features_test_in_use'

features_test = pd.read_csv(os.path.join(folder_test, os.listdir(folder_test)[0]))
for i in range(1,len(os.listdir(folder_test))):
    f = os.path.join(folder_test,os.listdir(folder_test)[i])
    if os.path.isfile(f):
        features_test = pd.concat([features_test, pd.read_csv(f)], axis=1)
print(features_test)
    

      plaace_h_1  plaace_h_2  plaace_h_3  plaace_h_4  features
0              1           1           1           0         0
1              1           1           1           0         0
2              1           1           1           0         0
3              1           1           1           0         0
4              1           1           1           0         0
...          ...         ...         ...         ...       ...
8572           2           8          11           2        85
8573           2           8          11           2        85
8574           2           8          11           2        85
8575           2           8          11           2        85
8576           2           8          11           2        85

[8577 rows x 5 columns]


Target train data:

In [6]:
target_train = pd.read_csv("../data/stores_train.csv")
target_train = target_train['revenue'].values

#log transform
target_train = np.log1p(target_train)

Light GBM model:

In [7]:
lgbm_model = lightgbm.LGBMRegressor(
    num_leaves=4,
    max_depth=5, 
    random_state=42, 
    silent=True, 
    metric='mse',
    n_jobs=4, 
    n_estimators=1000,
    colsample_bytree=0.95,
    subsample=0.9,
    learning_rate=0.05
)

In [8]:
#print(features_train)
#print(type(features_train))
print(len(features_train))
print(target_train)
print(type(target_train))
target_train = target_train
print(len(target_train))

12859
[2.94433371 3.21197205 2.83901998 ... 3.6693143  1.53514531 1.20237152]
<class 'numpy.ndarray'>
12859


In [9]:
lgbm_model.fit(features_train, target_train)
lgbm_predictions = lgbm_model.predict(features_test)

#Undo log transform
lgbm_predictions = np.expm1(lgbm_predictions)

c:\Python310\lib\site-packages\lightgbm\sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


In [10]:
print(lgbm_predictions)

[6.92478661 6.92478661 6.92478661 ... 3.25299387 3.25299387 3.25299387]


In [11]:
stores_test = pd.read_csv("../data/stores_test.csv")
stores_test_id = stores_test['store_id'].values

In [12]:
predictions = pd.DataFrame()

predictions['id'] = stores_test_id
predictions['predicted'] = lgbm_predictions
predictions.to_csv("../predictions/lightGBM7.csv", index=False)
print(predictions)


                              id  predicted
0     914206820-914239427-717245   6.924787
1     916789157-916823770-824309   6.924787
2       913341082-977479363-2948   6.924787
3      889682582-889697172-28720   6.924787
4     997991699-998006945-417222   6.924787
...                          ...        ...
8572  917323003-917383529-844309   3.252994
8573  917353379-917411824-845904   3.252994
8574  917072302-917089248-833647   3.252994
8575  916960557-916993161-829908   3.252994
8576   987280891-972040746-45320   3.252994

[8577 rows x 2 columns]


In [13]:
def rmsle(y_true, y_pred):
    """
    Computes the Root Mean Squared Logarithmic Error 
    
    Args:
        y_true (np.array): n-dimensional vector of ground-truth values 
        y_pred (np.array): n-dimensional vecotr of predicted values 
    
    Returns:
        A scalar float with the rmsle value 
    
    Note: You can alternatively use sklearn and just do: 
        `sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5`
    """
    assert (y_true >= 0).all(), 'Received negative y_true values'
    assert (y_pred >= 0).all(), 'Received negative y_pred values'
    assert y_true.shape == y_pred.shape, 'y_true and y_pred have different shapes'
    y_true_log1p = np.log1p(y_true)  # log(1 + y_true)
    y_pred_log1p = np.log1p(y_pred)  # log(1 + y_pred)
    return np.sqrt(np.mean(np.square(y_pred_log1p - y_true_log1p)))


X_train, X_test, y_train, y_test = train_test_split(features_train, target_train, test_size=0.2, random_state=42)

lgbm_model2 = lightgbm.LGBMRegressor(
    num_leaves=4,
    max_depth=5, 
    random_state=42, 
    silent=True, 
    metric='mse',
    n_jobs=4, 
    n_estimators=1000,
    colsample_bytree=0.95,
    subsample=0.9,
    learning_rate=0.05
)

lgbm_model2.fit(X_train, y_train)
y_pred = lgbm_model2.predict(X_test)
# Calculate rmsle for a few example predictions 
y_true = y_test
n = len(features_train)
print('A couple of RMSLE scores computed over the train set')
print(f'Perfect prediction: {rmsle(y_true, y_true):.4f}')
print(f'Score: {rmsle(y_true, y_pred):.4f}')

print("Score: ",sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5)

c:\Python310\lib\site-packages\lightgbm\sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


A couple of RMSLE scores computed over the train set
Perfect prediction: 0.0000
Score: 0.3467
Score:  0.3466855773904046
